## Handling "exceptions"

When you're programming, a lot can happen that you don't expect. You'll encounter a few different sets of errors:

* Things that are your fault (e.g. you had a typo)
* Things that aren't your fault, but that are under your control (you expected a CSV to be in Dropbox that someone else on your team deleted)
* Things that aren't your fault, but *not* under your control (you're using the Google Maps API to pull data, but Google returned an error)

**In Python, these errors are called "exceptions"** (the behavior is an _exception_ to normal behavior). You can write your code _without_ handling exceptions, but your programs will fail miserably. We'll see examples of such failures below. 

Your program may not be able to run without that CSV, or without hitting Google's API, so failing miserably might be OK in certain circumstances. But often, **just having your program fail isn't what you want**. If Google's API goes down, you may want to try hitting it again in 30 seconds. If that CSV you expect to be there isn't, you may want your script to send you an email letting you know.

### Objectives

* We'll start by reviewing the format of exceptions themselves, so you know how to read them when you encounter them.
* **The simplest way to catch and handle exceptions is with a `try`/`except` block**. We'll review how to handle simple exceptions using this technique.

### Objective 1: Understanding Exceptions

To handle exceptions, you need to be able to interpret them.

First, let's create an empty dictionary, and then attempt to reference the value at key 'a' **before we set a value at that key**.

In [46]:
dict = {}
print("The value at key 'a' is: %s" % dict['a'])

KeyError: 'a'

At first glance, this looks difficult to interpret. But there's a few key things Python is telling us that will help us 1.) understand the error and 2.) handle it.

The very first word in that error is **`KeyError`**. `KeyError` is the type of exception that Python has "raised" here ("raising" an exception just means the exception occurred). 

A "Traceback" follows: the set of code that caused the error. There's a big arrow pointing to the line that generated our error:

    ----> 2 print("The value at key 'a' is: %s" % dict['a'])
    
So we know our error happened somewhere in line 2.

We get a final bit of information specific to our exception at the bottom of this message. As the name indicates, `KeyError` is an error related to keys. Here, the only place we're using a key is when we try to reference the value of key 'a' in our dictionary dict. And that final line tells us the key where we received the `KeyError`:

    KeyError: 'a'
    
It's a `KeyError` on the key 'a'. Pretty quickly, we can scan back through our code and we realize that **we've forgotten to set a value at key 'a'**!

Let's take a look at another exception:

In [47]:
li = []
print("The first element of our list is: %s" % li[0])
print("We want this code to run, but we'll never get here")

IndexError: list index out of range

We learned a little bit about the format of our exception names above: if `KeyError` referred to an error with our use of keys, **`IndexError` refers to errors with our use of an index**.

Let's look at the traceback: this is happening on line 2, again, and the message is telling us our list index is "out of range". The only index we're using here is `0`, when we try to print the first element of the list. Again, **we haven't added any values to our list, so there is no value at that index**.

**So much of reading exceptions involves understanding the basic language around the data types we're working with**: key, index, etc. Once you grasp this language, reading these messages can be easier than they look.

It's worth noting that we had a final line of code in the cell above:

    print("We want this code to run, but we'll never get here")
    
This never executed. **Remember: When exceptions occur, the whole program fails miserably, and the remaining code fails to run**.

### Objective 2: Using try/except to handle exceptions

Clearly, the examples above were trivial errors. Each of those were bugs in our code, and within our control to fix.

Real-world errors are rarely this simple. Often, you're dealing with data users give you, or fetching data from external APIs, where many things can and will go wrong. It's not your own code that's the problem. Obviously, your code is perfect. Unexpected issues appear that cause your perfect code to fail.

Let's work with our dictionary example above. This time, we'll "handle" our `KeyError` exception with a `try`/`except` block:

In [48]:
dict = {}

# Put your code within the 'try' block
try:
    ## You fetch data from the Google Maps API and add it to your dictionary 'dict'.
    ## You expect to have some data at key 'a'. 

    print("The value at key 'a' is: %s" % dict['a'])
    
# If you encounter _any_ exception in the code within the try block, 
# the code will "jump" to the code within the except block. Here,
# that means we'll print "Something went wrong!"
except:
    print("Something went wrong!")
    
print("But our program keeps running...")

Something went wrong!
But our program keeps running...


As written, we have no value at the key 'a' in our dictionary (as above). This triggers a `KeyError`. But we don't see a `KeyError` message here. Instead, we print the message "*Something went wrong!*".

A `try`/`except` block works like this:

* Put the code you want to run within the `try` block.
* If you encounter _any_ exception (a `KeyError`, an `IndexError`, etc.), the code within the `try` block stops executing at the line where it failed, and Python immediately executes the code within the `except` block.

The keywords `try` and `except` come as a pair: you cannot have one without the other.

**Note: since we "handled" our `KeyError` exception, the rest of our program kept running**. This is the whole point of exception handling: you retain control of how your program handles the error. You can keep moving if you want. Or you can still fail miserably, if it doesn't make sense to continue based on the error.

Here, we handle our error with a vague message: "*Something went wrong!*". Typically, you'd want to handle this error in a better way. How can we improve this?

First, **you may want to print the error that occurred so you can investigate it later**. Here's how we do that:

In [49]:
dict = {}
try:
    print("The value at key 'a' is: %s" % dict['a'])
except Exception as e:
    print("Something went wrong: %s" % repr(e))

Something went wrong: KeyError('a',)


Here's how this works:

* `except Exception as e` captures any exception, naming our exception object `e`. This means we can use `e` to reference the exception that occurred within our `except` block.
* `repr(e)` prints a string representation of our exception, which contains both the exception that occurred (`KeyError`) and the key that caused the exception ('a')

To re-iterate our key takeways so far: 

* Keep the normal code you want to run within your `try` block.
* If anything goes wrong with that code, and an exception is "raised" (i.e some error happens), and no more code within that `try` block is run past the line that failed. 
* Then, the code within the `except` block is run.
* Any code that follows the `except` block is also run - the program continues unless you explicitly decide to exit.

So far, we've been handling **any** exception (`KeyError`, `IndexError`, etc.) within our `except` block. No matter the error, the code within our `except` block is run. What happens if we want to handle different errors differently?

In [50]:
try:
    dict = {}
    print("The value at key 'a' is: %s" % dict['a'])
except KeyError as e:
    print("It looks like there's no key %s in our dictionary" % str(e))
except IndexError as e:
    print("It looks like there's no element at index %s in our list" % str(e))
except Exception as e:
    print("Some other error occurred: %s"% repr(e))

It looks like there's no key 'a' in our dictionary


`except KeyError as e` allows us to handle `KeyError` exceptions specifically within this block. We're telling Python: when I see a `KeyError` exception, run the code within this block.

`except IndexError as e` does the same thing for `IndexError` exceptions: when Python encounters an `IndexError`, specifically, it will **skip the KeyError section** and run the code within this block, instead.

`except Exception as e` comes last: `Exception` is a catch-all exception type. If any other exceptions occur, we'll execute the code within this final block.

You'll notice we're using `str(e)` to print the key 'a' in our `KeyError` exception. `str(e)` happens to return the _value_ of the exception, that is the key that caused Python to throw the `KeyError` exception. But this is simply how `KeyError` generates its exception message. This may not work for other exceptions (in fact, it _does not_ work for `IndexError`, as we'll see below).

Let's change the code slightly so that we throw an `IndexError`, instead:

In [51]:
try:
    li = []
    print("The first element of our list is: %s" % li[0])
except KeyError as e:
    print("It looks like there's no key %s in our dictionary" % str(e))
except IndexError as e:
    print("It looks like we received an IndexError: %s" % repr(e))
except Exception as e:
    print("Some other error occurred: %s"% repr(e))

It looks like we received an IndexError: IndexError('list index out of range',)


Here, we ran the code within our `IndexError` block, since we encountered an `IndexError` in the main code (within our `try` block).

Let's generate another exception that will not be caught by either our `KeyError` or `IndexError` blocks. **What code will be run in that case?**

In [52]:
try:
    # Remember that we cannot divide by 0
    1 / 0
except KeyError as e:
    print("It looks like there's no key %s in our dictionary" % str(e))
except IndexError as e:
    print("It looks like we received an IndexError: %s" % repr(e))
except Exception as e:
    print("Some other error occurred: %s"% repr(e))

Some other error occurred: ZeroDivisionError('division by zero',)


Remember that the `except Exception as e` block captures *any other* exception that we don't otherwise have specific `except` blocks to capture. **`except Exception as e` acts as a catch-all**.

There's one final thing to note about syntax errors. Look at what happens with the following block of code:

In [53]:
try:
    print "This is a syntax error"
except KeyError as e:
    print("It looks like there's no key %s in our dictionary" % str(e))
except IndexError as e:
    print("It looks like we received an IndexError: %s" % repr(e))
except Exception as e:
    print("Some other error occurred: %s"% repr(e))

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int "This is a syntax error")? (<ipython-input-53-db3454f5e0d8>, line 2)

**This raises a `SyntaxError` exception**, which isn't expected. Why doesn't our `except Exception as e` block catch this syntax error?

Before a Python program is *executed*, its code is *parsed*. Python looks at the code and says, "here's a `for` loop, here's an `if else` block, here's a `print` statement", etc. This is parsing: Python **lays out a plan for executing the code before it actually executes it**. The code must be well-structured for the plan to succeed. If Python notices any major errors - namely, any errors in the basic structure of the code - the parsing fails. These errors typically manifest themselves as syntax errors.

So the `SyntaxError` is raised above because it happens during the parsing stage, **before any code is ever executed**. The code within the `except Exception as e` block is never run, because **none of the code is run**.

Why can't Python catch other errors, like `KeyError` or `IndexError`, in the parsing stage, and raise these errors before the program is run? That's the topic of another article...

### Resources

[Exceptions built into the Python language (like KeyError, IndexError)](https://docs.python.org/3/library/exceptions.html)